# Pipeline de Ingesta de Documentos
## Ingesta de Documentos PDF con Embeddings y Tags Automáticos

Este notebook implementa un pipeline completo para:
1. Extraer texto de archivos PDF
2. Dividir en chunks procesables
3. Generar embeddings con Azure OpenAI
4. Clasificar automáticamente con tags relevantes usando LLM
5. Guardar metadatos enriquecidos en JSON

## 1. Configuración de Variables de Entorno y Dependencias

In [ ]:
import json
import os
import pathlib
from openai import AzureOpenAI
import pymupdf4llm
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List, Dict, Any

# Load environment variables
load_dotenv(override=True)

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)
CHAT_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT_CHAT"]
EMBEDDING_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT_EMBEDDING"]

# Define directories
data_dir = pathlib.Path.cwd() / "data"
data_dir.mkdir(exist_ok=True)

print(f"Librerías importadas correctamente")
print(f"Variables de entorno cargadas")
print(f"Directorio de datos: {data_dir}")
print(f"Modelo de chat: {CHAT_DEPLOYMENT}")
print(f"Modelo de embeddings: {EMBEDDING_DEPLOYMENT}")

## 2. Cargar y Extraer Texto de Archivos PDF

In [ ]:
# Find all PDF files in data directory
filenames = [f.name for f in data_dir.glob("*.pdf")]

print(f"\nArchivos PDF encontrados: {len(filenames)}")
for i, filename in enumerate(filenames, 1):
    print(f"   {i}. {filename}")

# Dictionary to store extracted text from each file
extracted_documents = {}

for filename in filenames:
    file_path = data_dir / filename
    print(f"\nProcesando: {filename}")
    
    try:
        # Extract text from PDF to markdown format
        md_text = pymupdf4llm.to_markdown(file_path)
        extracted_documents[filename] = md_text
        
        print(f"   Texto extraído: {len(md_text)} caracteres")
        print(f"   Preview: {md_text[:150]}...")
    except Exception as e:
        print(f"   Error al procesar: {e}")

## 3. Dividir Texto en Chunks

In [ ]:
# Configure text splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o",
    chunk_size=500,
    chunk_overlap=125
)

# Store all chunks
all_chunks_raw = {}

print(f"\nDividiendo documentos en chunks...")
for filename, md_text in extracted_documents.items():
    texts = text_splitter.create_documents([md_text])
    all_chunks_raw[filename] = texts
    
    print(f"   {filename}: {len(texts)} chunks")
    
    # Show first chunk as example
    if texts:
        print(f"      Ejemplo - Chunk 1: {texts[0].page_content[:100]}...")

print(f"\nChunking completado")

## 4. Clasificar Chunks con LLM y Extraer Tags

Se usará un modelo LLM para analizar automáticamente cada chunk y extraer tags relevantes de la lista predefinida.

**Tags disponibles:**
- introduccion
- barrios
- gastronomia
- museos
- eventos
- naturaleza
- vida_nocturna
- transporte
- excursiones

In [ ]:
# Define available tags
AVAILABLE_TAGS = [
    "introduccion",
    "barrios",
    "gastronomia",
    "museos",
    "eventos",
    "naturaleza",
    "vida_nocturna",
    "transporte",
    "excursiones"
]

def extract_tags_from_chunk(text: str) -> List[str]:
    """
    Use an LLM to extract relevant tags from chunk content.
    
    Args:
        text: Chunk content
    
    Returns:
        List of relevant tags
    """
    try:
        prompt = f"""Analiza el siguiente texto y AÑADE SOLO SI SE MENCIONA EXPLICITAMENTE algo relacionado con uno o mas de estos tags: 
{', '.join(AVAILABLE_TAGS)}

Texto: {text[:1000]}

Responde SOLO con una lista de tags en formato JSON, ejemplo: ["tag1", "tag2"]
Sin texto adicional, solo el JSON."""
        
        response = client.chat.completions.create(
            model=CHAT_DEPLOYMENT,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=100
        )
        
        # Parse JSON response
        response_text = response.choices[0].message.content.strip()
        tags = json.loads(response_text)
        
        # Validate that all tags are in the allowed list
        valid_tags = [tag for tag in tags if tag in AVAILABLE_TAGS]
        return valid_tags if valid_tags else ["introduccion"]
        
    except Exception as e:
        print(f"   Error al extraer tags: {e}")
        return ["introduccion"]

print("Función de extracción de tags definida")

## 5. Generar Embeddings con Azure OpenAI

In [ ]:
def generate_embedding(text: str) -> List[float]:
    """
    Generate an embedding for a text using Azure OpenAI.
    
    Args:
        text: Text to generate embedding for
    
    Returns:
        Embedding vector (list of floats)
    """
    try:
        response = client.embeddings.create(
            model=EMBEDDING_DEPLOYMENT,
            input=text
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"   Error al generar embedding: {e}")
        return []

print("Función de generación de embeddings definida")

## 6. Estructurar Datos con Metadatos

Crear la estructura completa con: id, content, category (nombre del archivo), source, tags, embedding

In [ ]:
all_chunks = []
chunk_counter = 1

print(f"\nProcesando chunks con metadatos...")

for filename, chunks in all_chunks_raw.items():
    print(f"   {filename}")
    
    for i, chunk in enumerate(chunks, 1):
        try:
            text_content = chunk.page_content
            
            # Extract tags using LLM
            print(f"      Chunk {i}/{len(chunks)}: extrayendo tags...", end="")
            tags = extract_tags_from_chunk(text_content)
            print(f" {tags}")
            
            # Generate embedding
            print(f"      Chunk {i}/{len(chunks)}: generando embedding...", end="")
            embedding = generate_embedding(text_content)
            print(f" OK")
            
            # Create chunk structure with all metadata
            chunk_data = {
                "id": f"{filename.replace('.', '_')}-{i}",
                "content": text_content,
                "category": filename.replace(".pdf", ""),  # Filename without extension
                "source": "pdf_ingestion",  # Document source
                "tags": tags,
                "embedding": embedding
            }
            
            all_chunks.append(chunk_data)
            chunk_counter += 1
            
        except Exception as e:
            print(f"      Error al procesar chunk {i}: {e}")

print(f"\nProcesamiento completado: {len(all_chunks)} chunks generados")

## 7. Guardar Chunks Procesados en JSON

In [ ]:
# Save processed chunks to a JSON file
output_file = data_dir / "rag_ingested_chunks.json"

print(f"\nGuardando chunks en {output_file}...")

try:
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(all_chunks, f, indent=4, ensure_ascii=False)
    
    print(f"Archivo guardado exitosamente")
    print(f"Total de chunks: {len(all_chunks)}")
    print(f"Tamaño del archivo: {output_file.stat().st_size / 1024:.2f} KB")
    
except Exception as e:
    print(f"Error al guardar archivo: {e}")

## Visualización de Resultados

Revisar ejemplos de chunks procesados

In [ ]:
import pandas as pd

# Show summary information
print("Resumen de Chunks Procesados:")
print("=" * 60)

if all_chunks:
    # Create dataframe for analysis
    df_summary = pd.DataFrame([
        {
            "ID": chunk["id"],
            "Categoría": chunk["category"],
            "Tags": ", ".join(chunk["tags"]),
            "Longitud": len(chunk["content"]),
            "Has Embedding": len(chunk["embedding"]) > 0
        }
        for chunk in all_chunks[:10]  # Show first 10
    ])
    
    print(df_summary.to_string(index=False))
    
    if len(all_chunks) > 10:
        print(f"\n... y {len(all_chunks) - 10} chunks más")
    
    print("\n" + "=" * 60)
    print(f"Total de chunks: {len(all_chunks)}")
    
    # Statistics by category
    categories = {}
    for chunk in all_chunks:
        cat = chunk["category"]
        categories[cat] = categories.get(cat, 0) + 1
    
    print(f"\nDistribución por categoría:")
    for cat, count in sorted(categories.items()):
        print(f"   {cat}: {count} chunks")
    
    # Tag statistics
    all_tags = {}
    for chunk in all_chunks:
        for tag in chunk["tags"]:
            all_tags[tag] = all_tags.get(tag, 0) + 1
    
    print(f"\nDistribución de tags:")
    for tag, count in sorted(all_tags.items(), key=lambda x: x[1], reverse=True):
        print(f"   {tag}: {count} veces")
else:
    print("No hay chunks para mostrar. Asegúrate de que hay archivos PDF en el directorio /data")

## Ejemplo de Chunk Completo

Visualizar la estructura detallada de un chunk procesado

In [ ]:
if all_chunks:
    # Show first chunk as example
    example_chunk = all_chunks[0]
    
    print("Ejemplo de Chunk Procesado:")
    print("=" * 60)
    print(f"ID: {example_chunk['id']}")
    print(f"Categoría: {example_chunk['category']}")
    print(f"Source: {example_chunk['source']}")
    print(f"Tags: {example_chunk['tags']}")
    print(f"\nContenido (primeros 300 caracteres):")
    print(f"{example_chunk['content'][:300]}...")
    print(f"\nEmbedding (primeros 5 valores): {example_chunk['embedding'][:5]}")
    print(f"Dimensión del embedding: {len(example_chunk['embedding'])}")
else:
    print("No hay chunks disponibles para mostrar")